<a href="https://colab.research.google.com/github/kutyadog/ai_notebooks/blob/main/gmail_api_test_cj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Created Aug 27, 2024

To get this to work i first went to
https://console.cloud.google.com/apis/dashboard

Make sure under Enable APIs and Services you have GMAIL API

In Credentials, click Create Credentials -> API KEYS
*   Then Edit that api key (3 dots)
*   API Restrictions, choose Gmail

Create Credentials -> Oauth Client ID
*   Application type: Desktop app (or whatever you need)
*   Save
*   Download oAuth client JSON, save as client_secret.json locally

Oauth Consent Screen -> Under "Test Users" click "ADDED USERS"
  

In [ ]:
!pip install openai
!pip install simplegmail


In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

gdrive_file_id = userdata.get('GDRIVE_GOOGLE_CLIENT_SECRET_KEY_GMAIL')

# download GMAIL client secret from a google drive, or upload one into top directory
!gdown {gdrive_file_id} -O client_secret.json



In [11]:
# @title Setup Gmail and gmail functions

from simplegmail import Gmail
from simplegmail.query import construct_query

gmail = Gmail(noauth_local_webserver=True)
# gmail.load_credentials('client_secret.json')

def check_email(query):
  labels = gmail.list_labels()
  messages = gmail.get_messages(query=construct_query(query))
  return messages

In [ ]:
# @title Check Email Now

# Construct our two queries separately
query_params_1 = {
    "newer_than": (1, "day"),
    "in": "inbox",
    # sender=['john@doe.com', 'jane@doe.com'],
    # "exclude_sender": ['groupupdates@facebookmail.com'],
    "category": "primary",
    # "unread": True,
    # "labels":[["Finance"], ["Homework", "CS"]]
}

messages = check_email(query_params_1)

for message in messages:
    print("-----" + message.subject)
    print("Preview: " + message.snippet)
    # print("Date: " + str(message.date))
    # print("Sender: " + str(message.sender))


In [13]:
messages[0].plain

'Big AI release, JavaScript updates, and new browser features\r\n\r\nHi Chris,\r\n\r\nHappy Friday! We’re back with *Frontend to Fullstack Monthly #28* , bringing you the most important updates, tutorials, and standout projects shaping web development this August.\r\n\r\n------------\r\nNews updates\r\n------------\r\n\r\n🤖 OpenAI has released GPT-5 ( https://openai.com/index/introducing-gpt-5/ ) , said to be their " best model yet for coding and agentic tasks " — you can see the developer-oriented intro here ( https://openai.com/index/introducing-gpt-5-for-developers/ ).\r\n\r\nHow best to create masonry layouts in CSS has been a point of debate for a good while now, but there\'s still plenty of positive momentum towards making it a reality. In fact, you can currently test things as they stand in both Chrome and Edge ( https://developer.chrome.com/blog/masonry-update ) , with feedback on the approach being sought.\r\n\r\nRachel Andrew has shared an update on recent additions to the we

In [ ]:
# @title openAi functions
import openai

def summarize_email(email_text):
  response = openai.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant that summarizes emails into a TINY BULLETED LIST from the recipients point of view."},
        {"role": "user", "content": f"Please summarize the following email:\n\n{email_text}"}
    ],
    max_tokens=1200,
  )
  # print(response.choices[0])
  return response.choices[0].message.content
  # return response['choices'][0]['message']['content']

# for message in messages:
#   summary = summarize_email(message.text)
#   print(f"Subject: {message.subject}\nSummary: {summary}\n")



In [ ]:
summary = summarize_email(messages[0].plain)
print(f"Subject: {messages[0].subject}\nSummary: {summary}\n")

Subject: Appt on Friday
Summary: - Should I bring medical charts to my appointment on Friday?
- Do I need to fill out any paperwork in advance? Did not receive any if required.
- Freddy's contact: 502.909.0968



In [ ]:
# response = client.chat.completions.create(
#   model="gpt-4o",
#   messages=[
#         {"role": "system", "content": "You are a helpful assistant that summarizes emails."},
#         {"role": "user", "content": f"Please summarize the following email:\n\n{email_text}"}
#     ],
#   max_tokens=300,
# )

# print(response.choices[0])